<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [4]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 70.9MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 62.0MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 65.8MB/s]
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 123MB/s]


In [5]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip


Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.feature_selection as fe
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.feature_selection import RFECV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
#import category_encoders as ce


In [242]:
# date as index
df_train = pd.read_csv('train_features.csv',index_col='date_recorded', 
              parse_dates=True, infer_datetime_format=True)
df_test = pd.read_csv('test_features.csv', index_col='date_recorded',
                     parse_dates=True, infer_datetime_format=True)
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [246]:
# id as index
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [247]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        df_train, df_label, test_size=0.25, random_state=42, shuffle=True)

X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape

((44550, 39), (14850, 39), (44550, 2), (14850, 2))

In [248]:
# majority baseline
# balanced data - logistic class_weight = 
#            (None, accuracy = 0.55),
#            (balanced,accuracy=0.39-0.4)


df_label['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [249]:
pd.set_option('display.max_columns', None)

#df_str=X_train_val.select_dtypes(exclude=['int','float']).copy()
df_str=X_train_val.select_dtypes(object).copy()
df_str.head(3)


,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33935,2011-03-08,Government Of Tanzania,Ce,Ccm,Wami / Ruvu,Kilangawageni,Pwani,Bagamoyo,Mbwewe,True,GeoData Consultants Ltd,VWC,Kwaruhombo wa,True,ksb,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
49654,2013-02-16,Dwsp,DWE,Mwamahonza,Lake Victoria,Mwamahala,Mwanza,Magu,Nkungulu,NaN,GeoData Consultants Ltd,WUG,NaN,False,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
39287,2011-08-13,Dasip,DASIP,Chanongu,Lake Victoria,Chanongu,Mwanza,Kwimba,Wala,True,GeoData Consultants Ltd,VWC,NaN,True,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump


In [0]:
# As observations increase, more features will fail to pass the threshold
# At 50,000+ observation, passing threshold drops to 0.1

#remove all features that are either one or zero (on or off) in more than 80% of the samples.
#threshold = 0.8 * (1- .8)

import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.5):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [251]:
X_dummies = pd.get_dummies(df_str.funder)

dummy_filter=variance_threshold_selector(X_dummies,0.01)
dummy_filter.shape

(44550, 18)

In [0]:
# same as variance threshold function
def noise_filter(df, col, threshold=0.01):
  X_dummies=pd.DataFrame()
  X_dummies[col]=df[col].value_counts(normalize=True)

  for noise_cat in X_dummies.loc[(X_dummies[col] < threshold)].index.values:
    df[col].replace(noise_cat, 'noise', inplace=True)

  #df_str['funder'].replace('0', 'noise', inplace=True)
  #df_str['funder'].replace(np.nan,'noise' ,inplace=True)

  return pd.get_dummies(df[col])

In [253]:
dummy_filter = noise_filter(df_str, 'funder' ,0.01)
dummy_filter=dummy_filter.drop(['0','noise'], axis=1)
dummy_filter.shape

(44550, 18)

In [254]:
# Numeric data only
#df_sample = df_train.sample(2000)
#y_sample = df_label[df_label.id.isin(df_sample.id)]

X_num = X_train_val._get_numeric_data()

X_num.shape, y_train_val.shape

((44550, 9), (44550, 2))

In [255]:
X_num.head(2)

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,,,,,,,,,
33935,20.0,330,38.123839,-6.087137e+00,0,6,1,80,1972
49654,0.0,0,0.000000,-2.000000e-08,0,17,1,0,0


In [256]:
dummy_filter.head(2)

,Danida,Dhv,District Council,Dwsp,Germany Republi,Government Of Tanzania,Hesawa,Kkkt,Ministry Of Water,Norad,Private Individual,Rwssp,Tasaf,Tcrs,Unicef,Water,World Bank,World Vision
id,,,,,,,,,,,,,,,,,,
33935,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
49654,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X_num=X_num.join(dummy_filter, how='outer')

In [261]:
X_num.head(2)

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,Danida,Dhv,District Council,Dwsp,Germany Republi,Government Of Tanzania,Hesawa,Kkkt,Ministry Of Water,Norad,Private Individual,Rwssp,Tasaf,Tcrs,Unicef,Water,World Bank,World Vision
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
33935,20.0,330,38.123839,-6.087137e+00,0,6,1,80,1972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
49654,0.0,0,0.000000,-2.000000e-08,0,17,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [266]:
X_num.shape, y_train_val.shape

((44550, 27), (44550, 2))

In [267]:
# Baseline metric - numeric only - test_score=0.55
X= X_num
y= y_train_val.status_group

scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None), X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",1.749648,0.026553,0.553027,0.549715
1,"LogisticRegression(C=1.0, class_weight=None, d...",1.847138,0.025475,0.550640,0.552862
2,"LogisticRegression(C=1.0, class_weight=None, d...",1.825625,0.025530,0.550744,0.552002


In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [20]:

import sklearn.feature_selection as fe

names = X_train_val.columns

logr_u = LogisticRegression(solver='lbfgs')
# rank all features, i.e continue the elimination until the last one
rfecv = fe.RFECV(logr_u)
rfecv.fit(X_train_val, y_train_val)

# Make negative so it works better for comparison to other scores after scaling
#rfe_u_score = pd.Series(data=(-1*rfe_u.ranking_), name='RFE_unbal', index=names)

rfecv.ranking_
print "Features sorted by their rank:"
print sorted(zip(map(lambda x: round(x, 4), rfecv.ranking_), names))


Features sorted by their rank:
[(1.0, 'latitude'), (2.0, 'longitude'), (3.0, 'district_code'), (4.0, 'region_code'), (5.0, 'num_private'), (6.0, 'population'), (7.0, 'construction_year'), (8.0, 'gps_height'), (9.0, 'amount_tsh')]


In [268]:
# Define an estimator and param_grid
X = X_num
y = y_train_val.status_group

pipe = make_pipeline(
    MinMaxScaler(), 
    fe.SelectKBest(f_classif), 
    LogisticRegression(solver='lbfgs'))

param_grid = {
    'selectkbest__k': ['all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C': [.01, .1, 1.0, 10.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)

gs.fit(X, y)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head()

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   32.4s finished


()
('Cross-Validation Score:', 0.5804938271604938)
()
('Best estimator:', Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selectkbest', SelectKBest(k='all', score_func=<function f_classif at 0x7f86618caed8>)), ('logisticregression', LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]))
()
Best result from grid search of 8 parameter combinations


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
6,1.779259,0.039610,0.580494,0.581246,10,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",1,0.582587,0.579380,0.577172,0.583502,0.581723,0.580856,0.059919,0.000185,0.002375,0.001705
4,1.528937,0.030091,0.579506,0.580314,1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",2,0.580971,0.577562,0.576700,0.583468,0.580847,0.579913,0.056427,0.000558,0.001985,0.002428
2,0.847557,0.029742,0.578923,0.579428,0.1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",3,0.580500,0.576417,0.575892,0.583300,0.580376,0.578566,0.026084,0.001415,0.002143,0.002875
0,0.499284,0.025646,0.573558,0.574242,0.01,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",4,0.573968,0.572006,0.571044,0.575118,0.575662,0.575604,0.018950,0.004733,0.001907,0.001594
7,1.964829,0.041479,0.536364,0.536532,10,balanced,all,"{u'selectkbest__k': u'all', u'logisticregressi...",5,0.540435,0.536651,0.530909,0.535556,0.537747,0.537389,0.090316,0.000666,0.004010,0.000753


In [269]:
#Which features were selected?
X=X_num
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X.columns.values
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


Features selected:
amount_tsh
gps_height
longitude
latitude
num_private
region_code
district_code
population
construction_year
Danida
Dhv
District Council
Dwsp
Germany Republi
Government Of Tanzania
Hesawa
Kkkt
Ministry Of Water
Norad
Private Individual
Rwssp
Tasaf
Tcrs
Unicef
Water
World Bank
World Vision


Features not selected:


In [277]:
y.head()

AttributeError: ignored

In [280]:
y

array([1, 2, 2, ..., 1, 2, 1])

In [281]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

X=X_num
y=y_train_val.status_group

#y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
#y = y.values

X_scaled = RobustScaler().fit_transform(X)

# Lasso can't handle multi-class label.
#rfe = RFECV(Lasso(), scoring='accuracy', cv=3)
rfe = RFECV(LogisticRegression(), scoring='accuracy', cv=3)
X_subset = rfe.fit_transform(X_scaled, y)

all_names = X.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)


Features selected:
amount_tsh
gps_height
longitude
latitude
num_private
region_code
district_code
population
construction_year
Danida
Dhv
District Council
Dwsp
Germany Republi
Government Of Tanzania
Hesawa
Kkkt
Ministry Of Water
Norad
Private Individual
Rwssp
Tasaf
Tcrs
Unicef
Water
World Bank
World Vision
()
Features not selected:


In [180]:
print('Best result from grid search of{}'.format(len(results)))


Best result from grid search of8


In [88]:
# Predict with X_test features
y_pred = gs.predict(X_test)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test, y_pred)
print('Test Score:', test_score)

('Test Score:', 0.5603367003367004)
